# Silver to Gold Processing

This notebook aggregates data from `silver/` to create analytical tables in `gold/`, uses Azure Machine Learning for high-value donor predictions, and writes to Azure Synapse Analytics SQL pool for Power BI reporting.

## Inputs
- `silver/<table>/` (e.g., `donations`, `crm_data`)
- AML endpoint for predictions

## Outputs
- `gold/<table>/` (e.g., `donation_trends`, `predicted_donations`)
- Synapse SQL pool tables

## Dependencies
- `pyspark`, `azure-ai-ml`, `azure-identity`, `requests`

## Environment
- Uses `.env` for Blob Storage, AML, and Synapse credentials

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, count, datediff, hour, round, when, floor, date_trunc, current_date
from pyspark.ml.feature import VectorAssembler
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
import requests
from dotenv import load_dotenv
import os
import json

# Load environment variables
load_dotenv()
AZURE_CONN_STR = os.getenv("AZURE_STORAGE_CONNECTION_STRING")
CONTAINER_NAME = os.getenv("CONTAINER_NAME")
AZURE_STORAGE_ACCOUNT_NAME = os.getenv("AZURE_STORAGE_ACCOUNT_NAME")
AZURE_SUBSCRIPTION_ID = os.getenv("AZURE_SUBSCRIPTION_ID")
AZURE_RESOURCE_GROUP = os.getenv("AZURE_RESOURCE_GROUP")
AZURE_ML_WORKSPACE = os.getenv("AZURE_ML_WORKSPACE")
AZURE_ML_ENDPOINT_URL = os.getenv("AZURE_ML_ENDPOINT_URL")
AZURE_ML_ENDPOINT_KEY = os.getenv("AZURE_ML_ENDPOINT_KEY")
SYNAPSE_WORKSPACE_NAME = os.getenv("SYNAPSE_WORKSPACE_NAME")
SYNAPSE_DATABASE = os.getenv("SYNAPSE_DATABASE")
SYNAPSE_USER = os.getenv("SYNAPSE_USER")
SYNAPSE_PASSWORD = os.getenv("SYNAPSE_PASSWORD")

if not all([AZURE_CONN_STR, CONTAINER_NAME, AZURE_STORAGE_ACCOUNT_NAME, AZURE_ML_ENDPOINT_URL, AZURE_ML_ENDPOINT_KEY, SYNAPSE_WORKSPACE_NAME]):
    raise ValueError("Missing environment variables. Check .env file.")

# Initialize Spark session
spark = SparkSession.builder \
    .appName("SilverToGold") \
    .config("spark.hadoop.fs.azure.account.auth.type", "SAS") \
    .config("spark.hadoop.fs.azure.sas.token.provider.type", "org.apache.hadoop.fs.azure.SimpleSasTokenProvider") \
    .config("spark.hadoop.fs.azure.sas.fixed.token", AZURE_CONN_STR) \
    .config("spark.sql.catalogImplementation", "hive") \
    .getOrCreate()

# Define blob storage paths
blob_base_path = f"wasbs://{CONTAINER_NAME}@{AZURE_STORAGE_ACCOUNT_NAME}.blob.core.windows.net"
silver_path = f"{blob_base_path}/silver"
gold_path = f"{blob_base_path}/gold"

# Synapse connection properties
synapse_jdbc = f"jdbc:sqlserver://{SYNAPSE_WORKSPACE_NAME}.sql.azuresynapse.net:1433;database={SYNAPSE_DATABASE};encrypt=true;trustServerCertificate=false;hostNameInCertificate=*.sql.azuresynapse.net;loginTimeout=30;"
synapse_properties = {
    "user": SYNAPSE_USER,
    "password": SYNAPSE_PASSWORD,
    "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

# Read silver tables
donations = spark.read.parquet(f"{silver_path}/donations")
projects = spark.read.parquet(f"{silver_path}/projects")
volunteer_shifts = spark.read.parquet(f"{silver_path}/volunteer_shifts")
campaigns = spark.read.parquet(f"{silver_path}/campaigns")
volunteers = spark.read.parquet(f"{silver_path}/volunteers")
beneficiaries = spark.read.parquet(f"{silver_path}/beneficiaries")
transactions = spark.read.parquet(f"{silver_path}/transactions")
crm_data = spark.read.parquet(f"{silver_path}/crm_data")

# Aggregations
donations_per_project = donations.join(projects, "project_id") \
    .groupBy("project_id", "project_name") \
    .agg(
        sum("amount").alias("total_donations"),
        count("*").alias("donation_count")
    )

volunteer_hours = volunteer_shifts.join(projects, "project_id") \
    .groupBy("project_id", "project_name") \
    .agg(sum("shift_duration_hours").alias("total_hours"))

donations_by_region = donations.join(projects, "project_id") \
    .groupBy("region") \
    .agg(
        sum("amount").alias("total_donations"),
        count("*").alias("donation_count")
    )

campaign_performance = donations.join(campaigns, "campaign_id") \
    .groupBy("campaign_id", "title", "target_amount") \
    .agg(
        sum("amount").alias("total_donations"),
        count("*").alias("donation_count")
    ) \
    .withColumn("percent_target_achieved", round((col("total_donations") / col("target_amount")) * 100, 2))

donor_activity = donations.groupBy("donor_id", "donor_name") \
    .agg(
        sum("amount").alias("total_donations"),
        count("*").alias("donation_count")
    ) \
    .orderBy(col("total_donations").desc())

volunteer_engagement = volunteer_shifts.join(volunteers, "volunteer_id") \
    .withColumn("age_group", 
        when(col("age").between(16, 25), "16-25")
        .when(col("age").between(26, 35), "26-35")
        .when(col("age").between(36, 45), "36-45")
        .when(col("age").between(46, 55), "46-55")
        .when(col("age").between(56, 65), "56-65")
        .when(col("age").between(66, 80), "66-80")
        .otherwise("Unknown")
    ) \
    .groupBy("age_group") \
    .agg(
        sum("shift_duration_hours").alias("total_hours"),
        count("*").alias("shift_count")
    )

beneficiary_demographics = beneficiaries.groupBy("aid_type") \
    .agg(
        count("*").alias("beneficiary_count"),
        round(avg("age"), 2).alias("average_age")
    )

transaction_success_rate = transactions.groupBy("payment_provider") \
    .agg(
        count("*").alias("total_transactions"),
        sum(when(col("status") == "Success", 1).otherwise(0)).alias("successful_transactions")
    ) \
    .withColumn("success_rate", round((col("successful_transactions") / col("total_transactions")) * 100, 2))

donation_trends = donations.groupBy(date_trunc("month", "donation_date").alias("month")) \
    .agg(
        sum("amount").alias("total_donations"),
        count("*").alias("donation_count")
    ) \
    .orderBy("month")

active_campaigns = campaigns.join(projects, "project_id", "left") \
    .filter(col("end_date") >= current_date()) \
    .groupBy("region") \
    .agg(count("*").alias("active_campaign_count"))

# CRM-based donor engagement score
donor_engagement = donations.join(crm_data, "donor_id") \
    .groupBy("donor_id", "donor_name", "email") \
    .agg(
        count("*").alias("contact_count"),
        max("last_contact_date").alias("last_contact")
    ) \
    .withColumn("engagement_score", 
        when(datediff(current_date(), col("last_contact")) < 30, 3)
        .when(datediff(current_date(), col("last_contact")) < 90, 2)
        .otherwise(1) * col("contact_count")
    )

# AML-based high-value donor predictions
donor_features = donations.groupBy("donor_id", "donor_name") \
    .agg(
        sum("amount").alias("total_donations"),
        count("*").alias("donation_count"),
        avg("amount").alias("avg_donation")
    )
try:
    assembler = VectorAssembler(inputCols=["total_donations", "donation_count", "avg_donation"], outputCol="features")
    feature_data = assembler.transform(donor_features)
    feature_json = feature_data.select("donor_id", "donor_name", "features").toJSON().collect()
    predictions = []
    for row in feature_json:
        row_dict = json.loads(row)
        features = row_dict["features"]
        headers = {"Authorization": f"Bearer {AZURE_ML_ENDPOINT_KEY}", "Content-Type": "application/json"}
        payload = {"input_data": features}
        response = requests.post(AZURE_ML_ENDPOINT_URL, json=payload, headers=headers)
        if response.status_code == 200:
            pred = response.json()
            predictions.append((row_dict["donor_id"], row_dict["donor_name"], float(pred["prediction"])))
        else:
            print(f"AML endpoint error: {response.text}")
    predicted_donations = spark.createDataFrame(
        predictions,
        ["donor_id", "donor_name", "high_value_likelihood"]
    ).join(donor_features, ["donor_id", "donor_name"])
except Exception as e:
    print(f"AML prediction error: {str(e)}. Skipping predicted_donations.")
    predicted_donations = None

# Write to gold layer (Blob Storage)
donations_per_project.write.mode("overwrite").parquet(f"{gold_path}/donations_per_project")
volunteer_hours.write.mode("overwrite").parquet(f"{gold_path}/volunteer_hours_per_project")
donations_by_region.write.mode("overwrite").parquet(f"{gold_path}/donations_by_region")
campaign_performance.write.mode("overwrite").parquet(f"{gold_path}/campaign_performance")
donor_activity.write.mode("overwrite").parquet(f"{gold_path}/donor_activity")
volunteer_engagement.write.mode("overwrite").parquet(f"{gold_path}/volunteer_engagement")
beneficiary_demographics.write.mode("overwrite").parquet(f"{gold_path}/beneficiary_demographics")
transaction_success_rate.write.mode("overwrite").parquet(f"{gold_path}/transaction_success_rate")
donation_trends.write.mode("overwrite").parquet(f"{gold_path}/donation_trends")
active_campaigns.write.mode("overwrite").parquet(f"{gold_path}/active_campaigns")
donor_engagement.write.mode("overwrite").parquet(f"{gold_path}/donor_engagement")
if predicted_donations:
    predicted_donations.write.mode("overwrite").parquet(f"{gold_path}/predicted_donations")

# Write to Synapse dedicated SQL pool
try:
    donations_per_project.write.jdbc(synapse_jdbc, "donations_per_project", mode="overwrite", properties=synapse_properties)
    donations_by_region.write.jdbc(synapse_jdbc, "donations_by_region", mode="overwrite", properties=synapse_properties)
    campaign_performance.write.jdbc(synapse_jdbc, "campaign_performance", mode="overwrite", properties=synapse_properties)
    donor_activity.write.jdbc(synapse_jdbc, "donor_activity", mode="overwrite", properties=synapse_properties)
    donation_trends.write.jdbc(synapse_jdbc, "donation_trends", mode="overwrite", properties=synapse_properties)
    donor_engagement.write.jdbc(synapse_jdbc, "donor_engagement", mode="overwrite", properties=synapse_properties)
    if predicted_donations:
        predicted_donations.write.jdbc(synapse_jdbc, "predicted_donations", mode="overwrite", properties=synapse_properties)
except Exception as e:
    print(f"Synapse write error: {str(e)}.")

spark.stop()
print("Silver to Gold processing completed.")